![logo.png](logo.png) 


## Introduction
Welcome to Granular's Code Review Technical Challenge! 

This task involves commenting on a piece of code and its output. We expect it to take you at most **2 hours** of work. Your written answer should be **500-1000 words** (entries over 1000 words will be disqualified). To be clear, you will NOT be writing code, but rather reviewing it.

**Disclaimer:** The code in this exercise is not representative of code quality at Granular. We are presenting code with many errors and something like this would never go into production.

**Happy hunting!**

## Scenario

The file `toy_model_pipeline.py` contains Python code that loads historical data, trains models, and then uses those models to make predictions on a new dataset. `historical.csv` and `new_data.csv` contain train and test sets, respectively. They include simulated land sale data: each row describes a parcel of land. `y` is the sale price. `categorical_mapping.csv` maps the `category` feature to numerical features. `logfile.txt` contains output from running the script. `mse_by_model.png` visualizes the results. The goal is to predict prices as accurately as possible (interpretability is not a concern).

## Your task

Pretend that `toy_model_pipeline.py` is a piece of production code and review it. You should consider **architecture, style, logic, and maintainability** in your commentary. Explain bugs and other errors and give suggestions for improvement. No issue is too big or too small, and nothing is out of scope. Address any **TODO's** you see. When commenting on a specific function, or a specific line of code, please make that clear.  For example, you could structure your feedback like this:

- Lines 77-79: There is a bug related to XYZ. A sentence or two of explanation.

- Lines 82-88: Using parameters ABC for model DEF is a terrible idea. I would use parameter X because Y.

- Lines 203-204: Brittle/copy-pasted code. Needs a function.

- Overall, the code is difficult to maintain because of W. If I were responsible for this code, I'd recommend doing H.


## Hints and further guidance

This task is open-ended, and you should not feel constrained by what follows, but it may be helpful if you are unsure of where to start.

The output of `toy_model_pipeline.py` is puzzling. Look at the plot of model MSEs (mean squared errors) in `mse_by_model.png`: it appears that the models do worse on new data (x=2) than they do on held-out test data (x=1). Is that just a statistical fluke? Also, it looks like the random forest does worse than the linear model. What's going on?

Comment on any other interesting things you see in `mse_by_model.png` (including suggestions for improving the graph), in addition to commenting on the code itself. Your answer may include code snippets if you think that would be useful, but snippets are not required. You are free to run the code yourself as a way of exploring what it does. In case you are unable to run `toy_model_pipeline.py`, we have saved its output for you in `logfile.txt`.


In [ ]:
!git clone --branch granular_1 https://github.com/interviewquery/takehomes.git
%cd takehomes/ granular_1
!if [[ $(ls *.zip) ]]; then unzip *.zip; fi
!ls

In [ ]:
# Write your code here

The file `toy_model_pipeline.py` is made using Python 2 and needs to be reformatted to Python 3.

In [ ]:
#!/usr/bin/env python2.7
import matplotlib
matplotlib.use('Agg')

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def load_data():
    h = pd.read_csv('historical.csv')
    n = pd.read_csv('new_data.csv')
    categorical_mapping = pd.read_csv('categorical_mapping.csv')
    return h, n, categorical_mapping

def train_models(data, categorical_mapping,
                 abs_correlation_cutoff=0.15,
                 test_fraction=0.30,
    random_state=5123):
    assert 0.0 < abs_correlation_cutoff < 1.0
    assert 0.0 < test_fraction < 1.0
    print "training models (random forest, linear regression, mean model)"    
    random_forest = RandomForestRegressor(n_estimators=10,
                                          criterion="mse",
                                          max_features=None,
                                          max_depth=95,
                                          bootstrap=False,
                                          oob_score=False,
                                          random_state=random_state,
                                          verbose=0)
    linear_reg = LinearRegression(fit_intercept=True)

    print "head of data:"
    print data.head(3)
    print "head of categorical mapping (mapping from category string to additional X values):"
    print categorical_mapping.head(3)    
    ## Want to extend X matrix to include values in categorical_mapping.csv
    # data = pd.concat(data, categorical_mapping)  # Get predictor variables associated with data["category"]
    # np.mean(pd.isnull(data))
    # data.shape  # TODO Something is wrong, number of rows increased and there are lots of NAs, what's going on?
    
    X, y = data.drop('y', axis=1), data['y'].values.flatten()
    print "head of X:"
    print X.head(3)
    
    print "selecting Xs that are correlated with y (abs correlation > {})".format(abs_correlation_cutoff)
    corr_y_X = data.corr()['y'][:-1]
    predictor_indices = np.where(np.abs(corr_y_X) > abs_correlation_cutoff)[0]
    predictor_indices_string = ", ".join([str(x) for x in predictor_indices])                                         
    print " selected {} predictors (columns of X): indices {}".format(len(predictor_indices),
                                                                      predictor_indices_string)
        
    X_selected = X.iloc[:, predictor_indices]
    X_selected_train, X_selected_test, y_train, y_test = train_test_split(X_selected,
                                                                          y,
                                                                          test_size=test_fraction,
                                                                          random_state=random_state)

    print "fitting random forest to training data"
    random_forest.fit(X=X_selected_train, y=y_train)
    print "random forest feature importance (top 20 predictors by importance):"
    importance = pd.DataFrame({"importance":random_forest.feature_importances_,
                               "predictor_index":predictor_indices,
                               "predictor_name":X.columns[predictor_indices],
                               "corr_with_y":np.array(corr_y_X)[predictor_indices]})
    print importance.sort_values("importance", ascending=False).head(20)

    print "fitting linear regression to training data"
    linear_reg.fit(X=X_selected_train, y=y_train)

    print "fitting mean model to training data (mean of training data)"
    mean_model = np.mean(y_train)
    
    rf_predictions_train = random_forest.predict(X=X_selected_train)
    rf_predictions_test = random_forest.predict(X=X_selected_test)

    mean_predictions_train = np.ones_like(rf_predictions_train) * mean_model
    mean_predictions_test = np.ones_like(rf_predictions_test) * mean_model

    linear_predictions_train = linear_reg.predict(X=X_selected_train)
    linear_predictions_test = linear_reg.predict(X=X_selected_test)

    mse_estimates = {"rf_train":mean_squared_error(y_train, rf_predictions_train),
                     "rf_test":mean_squared_error(y_test, rf_predictions_test),
                     
                     "linear_train":mean_squared_error(y_train, linear_predictions_train),
                     "linear_test":mean_squared_error(y_test, linear_predictions_test),
                     
                     "mean_model_train":mean_squared_error(y_train, mean_predictions_train),
                     "mean_model_test":mean_squared_error(y_test, mean_predictions_test)}

    mse_std_errors = {"rf_train":np.std((y_train - rf_predictions_train) ** 2) / np.sqrt(len(y_train)),
                      "rf_test":np.std((y_test - rf_predictions_test) ** 2) / np.sqrt(len(y_test)),
                      
                      "linear_train":np.std((y_train - linear_predictions_train) ** 2) / np.sqrt(len(y_train)),
                      "linear_test":np.std((y_test - linear_predictions_test) ** 2) / np.sqrt(len(y_test)),
                      
                      "mean_model_train":np.std((y_train - mean_predictions_train) ** 2) / np.sqrt(len(y_train)),
                      "mean_model_test":np.std((y_test - mean_predictions_test) ** 2) / np.sqrt(len(y_test))}
           
    return random_forest, linear_reg, mean_model, predictor_indices, mse_estimates, mse_std_errors

def get_predictions(random_forest, linear_reg, mean_model, predictor_indices, data):
    X_selected = data.iloc[:, predictor_indices]
    rf_predictions = random_forest.predict(X_selected)
    linear_predictions = linear_reg.predict(X_selected)
    mean_model_predictions = mean_model * np.ones_like(rf_predictions)
    return (rf_predictions, linear_predictions, mean_model_predictions)

def save_plot_mse_estimates(mse_estimates, mse_std_errors,
    outfile="mse_by_model.png"):
    fig, ax = plt.subplots(figsize=(20, 10))
    mse_train_test_new = {"random forest":[mse_estimates["rf_train"],
                                           mse_estimates["rf_test"],
                                           mse_estimates["rf_new_data"]],
                          "linear model":[mse_estimates["linear_train"],
                                          mse_estimates["linear_test"],
                                          mse_estimates["linear_new_data"]],
                          "mean model":[mse_estimates["mean_model_train"],
                                        mse_estimates["mean_model_test"],
                                        mse_estimates["mean_model_new_data"]]}
    std_errors = {"random forest":[mse_std_errors["rf_train"],
                                   mse_std_errors["rf_test"],
                                   mse_std_errors["rf_new_data"]],
                  "linear model":[mse_std_errors["linear_train"],
                                  mse_std_errors["linear_test"],
                                  mse_std_errors["linear_new_data"]],
                  "mean model":[mse_std_errors["mean_model_train"],
                                mse_std_errors["mean_model_test"],
                                mse_std_errors["mean_model_new_data"]]}
    for model, errors in mse_train_test_new.iteritems():
        plt.plot(range(len(errors)), errors, "-o", label=model, linewidth=1.5, ms=9.0)
        ax.errorbar(range(len(errors)), errors, yerr=std_errors[model], ls='none')

    plt.xlim(-0.2, len(errors) - 1 + 0.2)
    plt.xlabel("0 = train, 1 = test, 2 = new data")
    plt.ylabel("MSE")
    plt.title("Error Estimates (Mean Squared Error) by Model\nWith 95% Confidence Intervals")
    ax.grid(color='grey', linestyle='--', alpha=0.50)
    plt.legend(loc=4)
    plt.savefig(outfile)
    plt.close("all")
    return
    
def main():
    historical_data, new_data, categorical_mapping = load_data()
    n_obs, n_features = historical_data.drop('y', axis=1).shape
    print "Training data: {} observations of {} predictors.".format(n_obs, n_features)
    
    (random_forest,
     linear_reg,
     mean_model,
     predictor_indices,
     mse_estimates,
     mse_std_errors) = train_models(historical_data, categorical_mapping,
                                    test_fraction=0.25, abs_correlation_cutoff=0.10)

    (rf_predictions,
     linear_predictions,
     mean_model_predictions) = get_predictions(random_forest, linear_reg, mean_model,
                                               predictor_indices, new_data.drop('y', axis=1))

    mse_estimates["rf_new_data"] = mean_squared_error(new_data["y"], rf_predictions)
    mse_estimates["linear_new_data"] = mean_squared_error(new_data["y"], linear_predictions)
    mse_estimates["mean_model_new_data"] = mean_squared_error(new_data["y"], mean_model_predictions)

    sqrt_len_new_y = np.sqrt(len(new_data["y"]))
    mse_std_errors["rf_new_data"] = np.std((new_data["y"] - rf_predictions) ** 2) / sqrt_len_new_y
    mse_std_errors["linear_new_data"] = np.std((new_data["y"] - linear_predictions) ** 2) / sqrt_len_new_y
    mse_std_errors["mean_model_new_data"] = np.std((new_data["y"] - mean_model_predictions) ** 2) / sqrt_len_new_y
    
    corr_y_rf_prediction = np.corrcoef(new_data["y"], rf_predictions)[0, 1]
    print "corr(y, rf_predictions) on new data: {}".format(corr_y_rf_prediction)
    corr_y_linear_prediction = np.corrcoef(new_data["y"], linear_predictions)[0, 1]
    print "corr(y, linear_predictions) on new data: {}".format(corr_y_linear_prediction)       

    print "error estimates by model:"
    print " linear regression: train {}, test {}, new data {}".format(mse_estimates["linear_train"],
                                                                      mse_estimates["linear_test"],
                                                                      mse_estimates["linear_new_data"])
    print " random forest: train {}, test {}, new data {}".format(mse_estimates["rf_train"],
                                                                  mse_estimates["rf_test"],
                                                                  mse_estimates["rf_new_data"])
    print " mean model : train {}, test {}, new data {}".format(mse_estimates["mean_model_train"],
                                                                mse_estimates["mean_model_test"],
                                                                mse_estimates["mean_model_new_data"])

    save_plot_mse_estimates(mse_estimates, mse_std_errors)
    
    print "all done"

if __name__ == "__main__":
    main()